# NVDR Holder Service - Non-Voting Depositary Receipt Analysis

Track NVDR (Non-Voting Depositary Receipt) holders and understand this unique Thai market instrument.

## Overview

**What are NVDRs?**
- Non-Voting Depositary Receipts created by Thai NVDR Company Limited
- Carry same rights as ordinary shares **except voting rights**
- Commonly used by foreign investors to bypass foreign ownership limits
- Identified by "-R" suffix (e.g., PTT-R)

**Use Cases:**
- Track NVDR holder distribution
- Monitor Thai vs foreign NVDR ownership
- Analyze liquidity and free float
- Compare with ordinary shareholders

In [ ]:
!pip install settfex

In [ ]:
import asyncio
from settfex.services.set import get_nvdr_holder_data
import pandas as pd

print("Libraries imported!")

## Basic Usage

In [ ]:
# Fetch NVDR holder data (note: automatically adds -R suffix)
data = await get_nvdr_holder_data("MINT")

print(f"Symbol: {data.symbol}")  # Will show MINT-R
print(f"Total NVDR Holders: {data.total_shareholder:,}")
print(f"Book Close Date: {data.book_close_date}")
print(f"Percent Scriptless: {data.percent_scriptless}%")
print(f"\nTop 10 NVDR Holders:\n")

for holder in data.major_shareholders[:10]:
    thai_flag = " [Thai NVDR]" if holder.is_thai_nvdr else ""
    print(f"{holder.sequence}. {holder.name}{thai_flag}")
    print(f"   Nationality: {holder.nationality}")
    print(f"   Shares: {holder.share_amount:,} ({holder.percent_of_share:.2f}%)")
    print()

## Advanced Usage - NVDR vs Ordinary Share Comparison

In [ ]:
from settfex.services.set import get_shareholder_data

async def compare_nvdr_vs_ordinary(symbol: str):
    """
    Compare NVDR holders vs ordinary shareholders.
    """
    # Fetch both
    ordinary, nvdr = await asyncio.gather(
        get_shareholder_data(symbol),
        get_nvdr_holder_data(symbol)
    )
    
    print(f"Comparison for {symbol}:\n")
    print(f"{'Metric':<30} {'Ordinary':<15} {'NVDR'}")
    print("-" * 60)
    print(f"{'Total Holders':<30} {ordinary.total_shareholder:<15,} {nvdr.total_shareholder:,}")
    print(f"{'Major Shareholders Listed':<30} {len(ordinary.major_shareholders):<15} {len(nvdr.major_shareholders)}")
    
    if ordinary.free_float:
        print(f"{'Free Float %':<30} {ordinary.free_float.percent_free_float:<15.2f} N/A")
    
    # Top holder comparison
    print(f"\nTop Holder Ordinary: {ordinary.major_shareholders[0].name} ({ordinary.major_shareholders[0].percent_of_share:.2f}%)")
    print(f"Top Holder NVDR: {nvdr.major_shareholders[0].name} ({nvdr.major_shareholders[0].percent_of_share:.2f}%)")

await compare_nvdr_vs_ordinary("CPALL")

## Use Case: Thai vs Foreign NVDR Analysis

In [ ]:
# Analyze Thai NVDR vs Foreign NVDR
data = await get_nvdr_holder_data("PTT")

thai_nvdr = [h for h in data.major_shareholders if h.is_thai_nvdr]
foreign_nvdr = [h for h in data.major_shareholders if not h.is_thai_nvdr]

thai_pct = sum(h.percent_of_share for h in thai_nvdr)
foreign_pct = sum(h.percent_of_share for h in foreign_nvdr)

print(f"NVDR Ownership Breakdown for {data.symbol}:\n")
print(f"Thai NVDR Holders: {len(thai_nvdr)} ({thai_pct:.2f}%)")
print(f"Foreign NVDR Holders: {len(foreign_nvdr)} ({foreign_pct:.2f}%)")
print(f"\nTop Thai NVDR Holders:")
for h in thai_nvdr[:3]:
    print(f"  {h.name}: {h.percent_of_share:.2f}%")

## Next Steps

- **[Shareholder](06_shareholder.ipynb)** - Ordinary shareholder analysis
- **[Stock Profile](03_stock_profile.ipynb)** - Foreign ownership limits

**Documentation**: `/docs/settfex/services/set/nvdr_holder.md`